In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json
import re

# Scraping dei personaggi dalla wiki Marvel

Ottenimento link delle rispettive pagine dei personaggi presenti nella wiki

In [ ]:
# main page with letters

next_link = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters"

characters_link_list = []
while next_link != None:
    response = requests.get(next_link)
    html_page = response.text
    html_tree = BeautifulSoup(html_page)
    results = html_tree.find_all("a", class_="category-page__member-link")
    next_link = html_tree.find(
        "a", class_="category-page__pagination-next wds-button wds-is-secondary")
    next_link = next_link["href"] if next_link != None else None

    for result in results:
        characters_link_list.append(
            [result.text, "https://marvelcinematicuniverse.fandom.com"+result["href"]])


In [26]:
lista_personaggi = pd.DataFrame(characters_link_list, columns=["Nome", "Link"])

Funzioni varie per lo scraping

In [5]:
def cerca_bio(page_tree):
    biography_list = []
    biography_initial = page_tree.find_all(id="Biography")
    description_ended = False
    if(biography_initial):
        next_span = biography_initial[0].parent.find_next()

        while not description_ended:
            if next_span.name != "a" and next_span.name != "sup":
                biography_list.append(next_span.text)

            next_span = next_span.find_next()
            if next_span.name == "h2":
                description_ended = True
            biography_list = list(dict.fromkeys(biography_list))
            lista_personaggi.at[idx, "Biografia"] = biography_list
    return biography_list

In [23]:
def cerca_serie(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "tv series":
            tag_lista_film = element

    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [22]:
def cerca_film(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "movie":
            tag_lista_film = element
    
    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [ ]:
def cerca_relazione_recursive(list_item):
  if not list_item:
    return []
  next_items = list_item.find_all("li")
  if not next_items:
    return [list_item.text.split(" - ",1)]
  else:
    recursive_list = []
    for item in next_items:
      recursive_list += cerca_relazione_recursive(item)
    return recursive_list



: 

In [14]:
def cerca_relazione(page_tree):
  relationship_initial_tag = page_tree.find(id="Relationships")
  if relationship_initial_tag:
    rel_table = relationship_initial_tag.find_next("table")
    relation_list = []
    if rel_table:
      for ul_list in rel_table.find_all("ul"):
        for list_item in ul_list.find_all("li"):
          relation_list += cerca_relazione_recursive(list_item)
    return relation_list


### Per ogni link precedentemente trovato viene recuperata dalla pagine della wiki la biografia del personaggio, i film e le serie in cui è presente e le relazioni con altri personaggi

In [8]:
from IPython.display import clear_output
lista_personaggi = pd.read_csv("./Datasets/scraping_personaggi_completo.csv")

lista_personaggi["Biografia"] = None
lista_personaggi["Film"] = None
lista_personaggi["Serie"] = None
lista_personaggi["Relazioni"] = None

num_iterazioni = 0

for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
    if row["Biografia"] == None:
        character_page_response = requests.get(row["Link"])
        character_page_tree = BeautifulSoup(character_page_response.text)
        biografia = cerca_bio(character_page_tree)
        film = cerca_film(character_page_tree)
        serie = cerca_serie(character_page_tree)
        relazioni = cerca_relazione(character_page_tree)
        lista_personaggi.at[idx, "Biografia"] = json.dumps(biografia)
        lista_personaggi.at[idx, "Film"] = json.dumps(film)
        lista_personaggi.at[idx, "Serie"] = json.dumps(serie)
        lista_personaggi.at[idx, "Relazioni"] = json.dumps(relazioni)


100%|██████████| 3746/3746 [03:49<00:00, 16.29it/s]   


Salvataggio dei risultati ottenuti

In [9]:
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo.csv", index=False)

In [11]:
lista_personaggi = pd.read_csv("./Datasets/scraping_personaggi_completo.csv")


,Nome,Link,Biografia,Relazioni,Processed_Name,Film,Serie
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Life on Titan"", ""A'Lars fathere...",[],A Lars,"[[""Avengers: Infinity War"", ""/wiki/Avengers:_I...",[]
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""First Encounter With S.H.I.E.L....",[],A. Kennedy,[],"[[""Agents of S.H.I.E.L.D."", ""/wiki/Agents_of_S..."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Obadiah Stane contacted the A.I...",[],A.I.M. President,[],[]
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Apple Inc."", ""Assisting Custome...",[],Aaron (Apple Inc.),"[[""Captain America: The Winter Soldier"", ""/wik...",[]
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Sophie walked with her son crad...",[],Aaron (Baby),[],"[[""Jessica Jones"", ""/wiki/Jessica_Jones_(TV_se..."
5,Aaron James,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Court Case"", ""With Matt Murdock...","[[""Father""], [""Mother""], [""Matt Murdock"", ""Law...",Aaron James,[],"[[""The Defenders"", ""/wiki/The_Defenders""], [""T..."
6,Aaron Martinez,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Aaron Martinez was a criminal, ...",[],Aaron Martinez,[],"[[""Jessica Jones"", ""/wiki/Jessica_Jones_(TV_se..."
7,Abby,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Terrigenesis"", ""\n\""When the Mi...","[[""Jemma Simmons""], [""Tye \u2020"", ""Trainer""],...",Abby,[],"[[""Agents of S.H.I.E.L.D."", ""/wiki/Agents_of_S..."
8,Abby Rose,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""To be added\n"", ""To be added""]",[],Abby Rose,[],[]
9,Abe Brown,https://marvelcinematicuniverse.fandom.com/wik...,"[""Biography"", ""Decathlon Team"", ""\n\""Fort Sumt...",[],Abe Brown,"[[""Spider-Man: Homecoming"", ""/wiki/Spider-Man:...",[]


# Data Cleaning



Normalizzazione dei nomi per efettuare un confronto più semplice con i nome di personaggi presenti nella web api

In [38]:
lista_personaggi["Processed_Name"] = None
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  lista_personaggi.at[idx, "Processed_Name"]= row["Nome"].replace("'", " ").replace('"', " ").replace(" \u2020","")
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo_filtered_name.csv", index=False)


100%|██████████| 3746/3746 [00:00<00:00, 21648.21it/s]


Rimozione caratteri speciali biografia e trasformazione in stringa semplice

In [41]:

lista_personaggi["Processed_Biography"] = None
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  for biography_item in json.loads(row["Biografia"]):
    if biography_item != "Biography":
      processed_biography_item = biography_item.replace(
          "\n", "").replace("'", "\\'").replace('"', '\\"').replace("[src]","").strip()
      processed_biography_item = re.sub("\[\d+\]","", processed_biography_item)
      if lista_personaggi.at[idx, "Processed_Biography"] != None:
        if processed_biography_item not in lista_personaggi.at[idx, "Processed_Biography"]:
          lista_personaggi.at[idx, "Processed_Biography"] = row["Processed_Biography"] + \
              "\n" + processed_biography_item if row["Processed_Biography"] != None else processed_biography_item
      else:
        lista_personaggi.at[idx, "Processed_Biography"] = row["Processed_Biography"] + \
            "\n" + \
            processed_biography_item if row["Processed_Biography"] != None else processed_biography_item
lista_personaggi.head()
lista_personaggi.to_csv(
    "./Datasets/scraping_personaggi_completo_filtered_name.csv", index=False)


100%|██████████| 3746/3746 [00:05<00:00, 643.25it/s]


### Data cleaning relazioni

Per orgni relazione trovata viene effettuato un controllo sui nomi dei personaggi rispetto ai personaggi ottenuti tramite scraping, se il nome non è trovato allora la relazione viene scartata

In [48]:

lista_personaggi = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_name.csv")
lista_personaggi["Filtered_relazioni"] = None
errors =0
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  try:
    relations = json.loads(row["Relazioni"])
    filtered_relations = []
    for relation in relations:
      splitted_relations = relation[0].split("/")
      for splitted_rel in splitted_relations:
        querystring = "Processed_Name == '%s'" % (
          relation[0].replace("'", " ").replace('"', " ").replace(" \u2020",""))
        if len(lista_personaggi.query(querystring)) != 0:
          filtered_relations.append([splitted_rel, relation[1]])

    lista_personaggi.at[idx, "Filtered_relazioni"] = json.dumps(filtered_relations)
  except Exception as e:
    errors+=1
print("Errors %s"%(errors))
  
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo_filtered_relations.csv", index=False)


100%|██████████| 3746/3746 [00:11<00:00, 323.13it/s]


Errors 1656


# Scraping info di film e serie tv

Funzioni per lo scraping di film

In [91]:
def get_film_info(page_tree):
  try:
    side_info = page_tree.find(
        "aside", class_="portable-infobox pi-background pi-border-color pi-theme-wikia pi-layout-default")
    divs = side_info.find_all(
        "div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    directors_list = []
    writer_list = []
    producer_list = []
    composer_list = []
    release_date = None
    runtime = None
    boxoffice = None
    for div in divs:
        if div["data-source"] == "director":
          directors_tags = div.find_all("a")
          for director in directors_tags:
            if director["href"] != "#cite_note-1":
              directors_list.append([director.text, director["href"]])
        elif div["data-source"] == "writer":
          writers_tags = div.find_all("a")
          for writer in writers_tags:
            if writer["href"] != "#cite_note-1":
              writer_list.append([writer.text, writer["href"]])
        elif div["data-source"] == "producer":
          producer_tags = div.find_all("a")
          for producer in producer_tags:
            if producer["href"] != "#cite_note-1":
              producer_list.append([producer.text, producer["href"]])
        elif div["data-source"] == "composer":
          composer_tags = div.find_all("a")
          for composer in composer_tags:
            if composer["href"] != "#cite_note-1":
              composer_list.append([composer.text, composer["href"]])
        elif div["data-source"] == "release":
          tags = div.find_all("div")
          for tag in tags:
            release_date = tag.text
        elif div["data-source"] == "runtime":
          tags = div.find_all("div")
          for tag in tags:
            runtime = tag.text
        elif div["data-source"] == "boxoffice":
          tags = div.find_all("div")
          for tag in tags:
            boxoffice = tag.text
    directors_list = json.dumps(directors_list)
    writer_list = json.dumps(writer_list)
    producer_list = json.dumps(producer_list)
    composer_list = json.dumps(composer_list)
    return ([page_tree.find(id="firstHeading").text.replace("\n",""), directors_list, writer_list, producer_list, composer_list, release_date, runtime, boxoffice])
  except Exception as e:
    print(e)

  

def get_synopsis(page_tree):
  try:
    synopsis = []
    title = page_tree.find(
        id="Synopsis")
    next_tag = title.find_next()
    while next_tag.name == "p" and next_tag:
      synopsis.append(next_tag.text)
      next_tag = next_tag.find_next_sibling()
    return (json.dumps(synopsis))
  except Exception as e:
    print(e)

  

def film_scraping(film_name="Avengers: Infinity War", film_url="https://marvelcinematicuniverse.fandom.com/wiki/Avengers:_Infinity_War"):
  try:
    film_page_response = requests.get(film_url)
    film_page_tree = BeautifulSoup(film_page_response.text)
    # film_info = get_film_info(film_page_tree)
    get_synopsis(film_page_tree)
  except Exception as e:
    print(e)


Ottenimento informazioni per ogni link di film ottenuto dallo scraping dei personaggi

In [92]:
lista_personaggi = pd.read_csv("Datasets/scraping_personaggi_completo_filtered_relations.csv")
lista_film_links = []
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  films = json.loads(row["Film"])
  for film in films:
    if "https://marvelcinematicuniverse.fandom.com"+film[1] not in lista_film_links:
      lista_film_links.append("https://marvelcinematicuniverse.fandom.com"+str(film[1]))
film_list = []
for film_link in tqdm(lista_film_links):
  try:
    film_page_response = requests.get(film_link)
    film_page_tree = BeautifulSoup(film_page_response.text)
    film_info = get_film_info(film_page_tree)
    film_synopsis = get_synopsis(film_page_tree)
    film_list.append(film_info+ [film_synopsis])
  except Exception as e:
    print(e)

 86%|████████▌ | 36/42 [00:07<00:01,  5.49it/s]

'NoneType' object has no attribute 'find_next'


 90%|█████████ | 38/42 [00:08<00:01,  3.20it/s]

'NoneType' object has no attribute 'find_next'


100%|██████████| 42/42 [00:09<00:00,  4.33it/s]


In [93]:
film_datafram = pd.DataFrame(film_list, columns=["Title","Directors", "Writers", "Producers","Composers","releaseDate","Runtime","BoxOffice","Synopsis"])
film_datafram.head()

,Title,Directors,Writers,Producers,Composers,releaseDate,Runtime,BoxOffice,Synopsis
0,Avengers: Infinity War,"[[""Joe Russo"", ""/wiki/Joe_Russo""], [""Anthony R...","[[""Christopher Markus"", ""/wiki/Christopher_Mar...","[[""Kevin Feige"", ""/wiki/Kevin_Feige""], [""Louis...","[[""Alan Silvestri"", ""/wiki/Alan_Silvestri""]]","April 27, 2018",149 minutes,$2.049 billion,"[""An unprecedented cinematic journey ten years..."
1,Captain America: The Winter Soldier,"[[""Joe Russo"", ""/wiki/Joe_Russo""], [""Anthony R...","[[""Stephen McFeely"", ""/wiki/Stephen_McFeely""],...","[[""Kevin Feige"", ""/wiki/Kevin_Feige""], [""Louis...","[[""Henry Jackman"", ""/wiki/Henry_Jackman""]]","March 26, 2014 (international)April 4, 2014 (U...",136 minutes,$714.4 million,"[""After the cataclysmic events in New York wit..."
2,Spider-Man: Homecoming,"[[""Jon Watts"", ""/wiki/Jon_Watts""]]","[[""Jonathan M. Goldstein"", ""/wiki/Jonathan_M._...","[[""Kevin Feige"", ""/wiki/Kevin_Feige""], [""Amy P...","[[""Michael Giacchino"", ""/wiki/Michael_Giacchin...","July 5, 2017 (U.K.)July 7, 2017 (U.S.)",133 minutes,$880.2 million,"[""A young Peter Parker/Spider-Man, who made hi..."
3,Guardians of the Galaxy Vol. 2,"[[""James Gunn"", ""/wiki/James_Gunn""]]","[[""James Gunn"", ""/wiki/James_Gunn""]]","[[""Kevin Feige"", ""/wiki/Kevin_Feige""], [""Louis...","[[""Tyler Bates"", ""/wiki/Tyler_Bates""]]","April 25, 2017 (International)May 5, 2017 (U.S.)",137 minutes,$863.8 million,"[""Set to the backdrop of \""Awesome Mixtape #2\..."
4,The Incredible Hulk,"[[""Louis Leterrier"", ""/wiki/Louis_Leterrier""]]","[[""Zak Penn"", ""/wiki/Zak_Penn""], [""Edward Nort...","[[""Avi Arad"", ""/wiki/Avi_Arad""], [""Gale Anne H...","[[""Craig Armstrong"", ""/wiki/Craig_Armstrong""]]","June 13, 2008",112 minutes,$263.4 million,"[""In this new beginning, scientist Bruce Banne..."


In [ ]:
film_datafram.to_csv("./Datasets/film_data_scraping.csv",index=False)

### Scraping serie tv
Funzioni per ottenere le informazioni delle serie

In [3]:
def get_serie_info(page_tree):
  try:
    side_info = page_tree.find(
        "aside", class_="portable-infobox pi-background pi-border-color pi-theme-wikia pi-layout-default")
    divs = side_info.find_all(
        "div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    creators_list = []
    showrunner_list = []
    producer_list = []
    composer_list = []
    release_date = None
    for div in divs:
        if div["data-source"] == "creator":
          creators_tags = div.find_all("a")
          for creator in creators_tags:
            if creator["href"] != "#cite_note-1":
              creators_list.append([creator.text, creator["href"]])
        elif div["data-source"] == "showrunners":
          showrunners_tags = div.find_all("a")
          for showrunner in showrunners_tags:
            if showrunner["href"] != "#cite_note-1":
              showrunner_list.append([showrunner.text, showrunner["href"]])
        elif div["data-source"] == "exproducers":
          producer_tags = div.find_all("a")
          for producer in producer_tags:
            if producer["href"] != "#cite_note-1":
              producer_list.append([producer.text, producer["href"]])
        elif div["data-source"] == "composer":
          composer_tags = div.find_all("a")
          for composer in composer_tags:
            if composer["href"] != "#cite_note-1":
              composer_list.append([composer.text, composer["href"]])
        elif div["data-source"] == "dates":
          tags = div.find_all("div")
          for tag in tags:
            release_date = tag.text
        
    creators_list = json.dumps(creators_list)
    showrunner_list = json.dumps(showrunner_list)
    producer_list = json.dumps(producer_list)
    composer_list = json.dumps(composer_list)
    return ([page_tree.find(id="firstHeading").text.replace("\n", ""), creators_list, showrunner_list, producer_list, composer_list, release_date])
  except Exception as e:
    print(e)


def get_synopsi_series(page_tree):
  try:
    synopsis = []
    string_title = ""
    title = page_tree.find(
        id="Season_One")
    if title:
      next_tag = title.find_next()
      while next_tag and (next_tag.name == "p" or next_tag.name == "a") and next_tag.text != "Season Two":
        string_title += "\n"+next_tag.text
        next_tag = next_tag.find_next_sibling()
      return string_title
    else:
      return None
  except Exception as e:
    print(e)


def film_scraping(film_name="Avengers: Infinity War", film_url="https://marvelcinematicuniverse.fandom.com/wiki/Avengers:_Infinity_War"):
  try:
    film_page_response = requests.get(film_url)
    film_page_tree = BeautifulSoup(film_page_response.text)
    # film_info = get_film_info(film_page_tree)
    get_synopsis(film_page_tree)
  except Exception as e:
    print(e)


### Ottenimento informazioni per ogni link di serie presente nella lista personaggi

In [4]:
lista_personaggi = pd.read_csv("Datasets/scraping_personaggi_completo_filtered_relations.csv")
lista_serie_links = []
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  series = json.loads(row["Serie"])
  for serie in series:
    if "https://marvelcinematicuniverse.fandom.com"+serie[1] not in lista_serie_links:
      lista_serie_links.append(
          "https://marvelcinematicuniverse.fandom.com"+str(serie[1]))
serie_list = []
for serie_link in tqdm(lista_serie_links):
  try:
    serie_page_response = requests.get(serie_link)
    serie_page_tree = BeautifulSoup(serie_page_response.text)
    serie_info = get_serie_info(serie_page_tree)
    serie_synopsis = get_synopsi_series(serie_page_tree)
    serie_list.append(serie_info + [serie_synopsis])
  except Exception as e:
    print(e)

100%|██████████| 437/437 [02:21<00:00,  3.09it/s]


### Data cleaning
Rimozioni dei singoli episodi delle serie TV in quanto non sono disponibili sulla wiki le info di tutti gli episodi.
I record da eliminare sono quelli che non presentano dati nel campo *Creators*

In [5]:
filtered_series = []
for serie in serie_list:
  if len(json.loads(serie[1])) != 0:
    filtered_series.append(serie)
serie_datafram = pd.DataFrame(filtered_series, columns=[
    "Title", "Creators", "Showrunners", "Producers", "Composers", "releaseDate", "Synopsis"])
serie_datafram.head(20)
serie_datafram.to_csv("./Datasets/tv_series.csv")
